In [23]:
import pandas as pd
import numpy as np
import pyspark.pandas as ps
# from pyspark.ml.linalg import Vectors

In [24]:
# Local mode
spark = SparkSession.builder.appName("sqldemo").getOrCreate()

In [ ]:
# standalone mode
spark = SparkSession\
        .builder\
        .master("spark://master.example.org:7077")\
        .config('spark.cores.max','1')\
        .config('spark.executor.memory','1G')\
        .appName("clusterdemo")\
        .getOrCreate()

In [25]:
# 開啟最佳化 Spark 效能
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", True)

***

In [26]:
'''
在python pd.DataFram.to_csv(index=False) 匯出.csv 格式，避免 inferScaeme 抓不到第一欄名稱報錯。
'''
# 讀取.csv 轉為 spark.sql.dataframe
print("=== spark.read.csv('allmusic3s_new.csv',sep=",",header=True, inferSchema=True) ===")
%time sdf_3s = spark.read.csv('allmusic3s_new.csv',sep=",",header=True, inferSchema=True)

# 創建資料庫
print("=================== sdf_3s.createOrReplaceTempView('\'dfTable'\') ====================")
%time sdf_3s.createOrReplaceTempView("dfTable")

# 刪除 NaN row
print("=================== sdf_3s.createOrReplaceTempView('\"dfTable'\') ====================")
%time sdf_3s = sdf_3s.dropna(how='any')

=== spark.read.csv('allmusic3s_new.csv',sep= ,header=True, inferSchema=True) ===


CPU times: user 6.37 ms, sys: 0 ns, total: 6.37 ms
Wall time: 1.01 s
=================== sdf_3s.createOrReplaceTempView(''dfTable'') ====================
CPU times: user 490 µs, sys: 0 ns, total: 490 µs
Wall time: 10.3 ms
=================== sdf_3s.createOrReplaceTempView('"dfTable'') ====================
CPU times: user 15.1 ms, sys: 0 ns, total: 15.1 ms
Wall time: 84.7 ms


In [6]:
# 刪除多餘欄位
sdf_cols = sdf_3s.columns
select_sdf_3s = sdf_3s.select(sdf_cols[1:])

# 抓取非字串欄位的 list，辨識非字串型別 column != "label"
continuous_features = [d[0] for d in select_sdf_3s.dtypes if (d[1] != 'string')]

***

In [14]:
from pyspark.ml.feature import StringIndexer, MinMaxScaler,VectorAssembler
from pyspark.ml import Pipeline

# label 編碼
# 選取需要的標籤 input="label", output="y"
indexers = [StringIndexer(inputCol="label", outputCol="y")] 

# 特徵轉向量
# input = for迴圈取出所有column 並[]轉成 list, output= "features" 一個標籤
assemblers = VectorAssembler(inputCols=[col for col in continuous_features], outputCol="features")

# MinMixScaler 標準化
# input= "features"(Vector column), output="mmfeartures"
mmScalers = MinMaxScaler(inputCol="features", outputCol="mmfeatures")

# Piplin整合一起執行
pipeline = Pipeline(stages= [assemblers, mmScalers] + indexers)
print("=========== pipeline.fit(select_sdf_3s) ============")
%time scalerModel = pipeline.fit(select_sdf_3s) 
print("======= scalerModel.transform(select_sdf_3s) =======")
%time scaledData = scalerModel.transform(select_sdf_3s)

#切分訓練、測試資料
data = scaledData.select("features","mmfeatures", "y")
trainingData, testData = data.randomSplit([0.8, 0.2])

=========== >>> pipeline.fit(select_sdf_3s)============


CPU times: user 7.84 ms, sys: 24.2 ms, total: 32 ms
Wall time: 2.48 s


====== >>> scalerModel.transform(select_sdf_3s) =======
CPU times: user 31 ms, sys: 0 ns, total: 31 ms
Wall time: 128 ms


In [ ]:
# 查看結構 
#select_sdf_3s.printSchema()

In [ ]:
# 查看訓練和測試筆數
# print("trainingData: {} count,testData: {} count".format(trainingData.count(), testData.count()))

***

## ML Part

In [16]:
from pyspark.ml.classification import LogisticRegression, RandomForestClassifier, GBTClassifier
def select_model(algo, train, test): #( 已建立模型物件, 訓練資料, 測試資料 )
    model = algo.fit(train)    # alog 傳入模型
    predictions = model.transform(test)   
    test_result = model.evaluate(test)
    print('{} Accuracy: {:5.3f}'.format(type(algo).__name__, test_result.accuracy)) # 列印準確度和模型名稱

In [17]:
# 建立空模
# LogisticRegression
logr = LogisticRegression(featuresCol='mmfeatures',labelCol='y')

# RandomForest
rf = RandomForestClassifier(featuresCol='mmfeatures',labelCol='y',numTrees=1000 ,maxDepth=10)

# GBTClassifier
gbt = GBTClassifier(featuresCol='mmfeatures',labelCol='y')

## Training model times

In [18]:
%time select_model(rf, trainingData, testData)

22/10/05 14:29:14 WARN DAGScheduler: Broadcasting large task binary with size 1844.7 KiB


22/10/05 14:29:29 WARN DAGScheduler: Broadcasting large task binary with size 3.7 MiB


22/10/05 14:29:45 WARN DAGScheduler: Broadcasting large task binary with size 1193.0 KiB


22/10/05 14:29:49 WARN DAGScheduler: Broadcasting large task binary with size 3.9 MiB


22/10/05 14:30:02 WARN DAGScheduler: Broadcasting large task binary with size 1193.0 KiB


...

22/10/05 14:36:11 WARN DAGScheduler: Broadcasting large task binary with size 1148.4 KiB


22/10/05 14:36:13 WARN DAGScheduler: Broadcasting large task binary with size 4.7 MiB


22/10/05 14:36:16 WARN DAGScheduler: Broadcasting large task binary with size 4.0 MiB


22/10/05 14:36:26 WARN DAGScheduler: Broadcasting large task binary with size 160.4 MiB


RandomForestClassifier Accuracy: 0.721
CPU times: user 269 ms, sys: 265 ms, total: 534 ms
Wall time: 7min 58s


In [ ]:
%time select_model(logr, trainingData, testData)

In [ ]:
%time select_model(gbt, trainingData, testData)